In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from mpl_toolkits.mplot3d import Axes3D
from heuristi_plot_1 import plot as hplot

path = '../output_data/'

In [ ]:
for i in [1, 3, 5, 6, 7, 8, 9]:
#for i in [5, 8, 9]:
    print(f'phi = 0.{i}')
    with pd.HDFStore(path + 'X0c.h5') as store:
        trj = store.select('dat_0', where=f'phi=.{i}')
    with pd.HDFStore(path + 'X0c.h5') as store:
        trj2 = store.select('dat_2', where=f'phi=.{i}')

    trj2.index.names = trj.index.names

    for trj_i in [trj, trj2]:
        trj_i.index = trj_i.index.droplevel(['eps', 'phi', 'test'])
        trj_i = trj_i.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])
    trj = trj.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])
    trj2 = trj2.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])

    dfN = pd.DataFrame(index=trj.index.levels[0])

    fig = plt.figure(figsize=(10, 3))
    hplot(fig, trj, trj2, thresh_time=9.)

    fig.savefig(f'fitted_initials_evaluation0{i}.pdf')

In [ ]:
cond = pd.DataFrame(index=trj.index)
cond['a'] = trj['K_c']/trj['K_d'] - 0.15276374882897656
cond['b'] = trj['i_c']
cond['c'] = trj['r_c']

# cond['d'] = np.sqrt((cond['a'])**2)
# cond['d'].unstack('sample').loc[:9].sum(axis=0)

In [ ]:
trj.head()

In [ ]:
t_max=9.

# calculate point distance
cond['d'] = np.sqrt((cond['a'])**2)

# sum over point distance for each trajectory
dfN['distance'] = cond['d'].unstack('sample').loc[:t_max].sum(axis=0)

# map distances to interval (0,1]
dfN['inv_distance'] = 1./(1. + dfN['distance'])

# use 1/distance as weight
dfN['weight'] = 1/cond['d'].unstack('sample').loc[:t_max].sum(axis=0)
dfN['weight'] = dfN['weight']/sum(dfN['weight'].values)

for s in dfN.index.values:
    cond.loc[s, 'inv_distance'] = dfN.loc[s, 'inv_distance']
dfN['weight'].sum()
cond.head()

In [ ]:
from collections import Counter
fig, axes = plt.subplots(ncols=3, figsize=(8, 2.8))

def plot_cond(df, c, ax, t_max, thresh=0.5):
    tmp = df
    tmp1 = tmp[[c]].loc[tmp['inv_distance']<thresh]
    tmp2 = tmp[[c]].loc[tmp['inv_distance']>thresh]
    tmp1.unstack('sample').loc[:t_max].plot(ax=ax, alpha=.03, legend=False, color=['k'])
    tmp2.unstack('sample').loc[:t_max].plot(ax=ax, legend=False, alpha=.07, color=['orange'])
    
thresh=0.5

plot_cond(cond, 'a', axes[0], 20, thresh=thresh)
plot_cond(cond, 'b', axes[1], 20, thresh=thresh)

dfN['inv_distance'].loc[dfN['inv_distance']>thresh].plot.hist(ax=axes[2], color=['orange'], alpha=0.9, bins=0.1*np.arange(5,11))
dfN['inv_distance'].loc[dfN['inv_distance']<thresh].plot.hist(ax=axes[2], color=['k'], alpha=0.2, bins=0.1*np.arange(0,6))

for ax in axes[:2]:
    ax.set_xlim([0,20])
    ax.set_xlabel('time')
    ax.axvline(t_max, color='k')

axes[0].set_ylabel(r'$K_c/K_d - K_{c0}/K_{d0}$')
axes[1].set_ylabel('fraction of clean investment')
axes[1].set_ylim((0,1.03))
axes[2].set_ylabel('count')
axes[2].set_xlabel(r'$1/(1+d)$')
axes[2].set_xlim([0,1])

fig.tight_layout()
fig.savefig('initial_condition_fitted.pdf')

In [ ]:
with pd.HDFStore(path + 'X0c.h5') as store:
    trj = store.select('dat_2', where='phi=.5')
with pd.HDFStore(path + 'X0c.h5') as store:
    trj2 = store.select('dat_2', where='phi=.5')
    
trj2.index.names = trj.index.names

for trj_i in [trj, trj2]:
    trj_i.index = trj_i.index.droplevel(['eps', 'phi', 'test'])
    trj_i = trj_i.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])


In [ ]:
data = trj.reset_index().groupby('time').mean()
data_err = trj.reset_index().groupby('time').std()
data

In [ ]:
fig, ax  = plt.subplots()

variables = ['K_c', 'K_d']
colors = ['r', 'b']

for i, var in enumerate(variables):
    t = data.index.values
    x = data[var].values
    dx = data_err[var].values
    ln = ax.plot(t, x, color=colors[i], label=var)
    ax.fill_between(t, x+dx, x-dx, color='k', alpha=0.2)
ax.set_xlim([0, 100])

In [ ]:
def plotfunc(variables, colors, data, data_err, ax):
    lns = []
    for i, var in enumerate(variables):
        
        t = data.index.values
        x = data[var].values
        dx = data_err[var].values
        
        ln = ax.plot(t, x, color=colors[i], label=var)[0]
        ax.fill_between(t, x+dx, x-dx, color='k', alpha=0.2)
        lns.append(ln)
        
        ax.set_xlim([0, 40])
    print(lns)
    return lns

In [ ]:
green  = '#69923A'
blue = '#009FDA'
grey = '#8E908F'
orange = '#E37222'

colors = [blue, orange]
fig = plt.figure(figsize=(7, 5))

# top left figure, r_c, r_d, n_c
ax1 = fig.add_subplot(221)
lns1 = plotfunc(variables=['r_c', 'r_d'], 
                colors=[blue, orange], 
                data=data, 
                data_err=data_err,
                ax=ax1)
ax1.set_ylim([0.03, 0.12])
ax1b = ax1.twinx()
lns1b = plotfunc(variables=['i_c'],
                colors=[grey], data=data, data_err=data_err, ax=ax1b)
ax1.set_ylabel('capital return r')
ax1b.set_ylabel('frac. clean investment i_c')
ax1.set_xticklabels([])

# top right figure: c, g
ax2 = fig.add_subplot(222)
lns2 = plotfunc(['C'], [blue], data, data_err, ax2)
ax2b = ax2.twinx()
lns2b = plotfunc(variables=['G'],
                colors=[orange], data=data, data_err=data_err, ax=ax2b)
ax2.set_ylim([0, 1.5e16])
ax2.set_ylabel('knowledge c')
ax2b.set_ylabel('resource stock g')
ax2.set_xticklabels([])

# bottom left figure: Y_c, Y_d
ax3 = fig.add_subplot(223)
lns3 = plotfunc(variables=['Y_c', 'Y_d'],
                colors=[blue, orange], 
                data=data,
                data_err=data_err,
                ax=ax3)
ax3.set_ylim([0, 7e14])
ax3.set_ylabel('Production Y')
ax3.set_xlabel('t')

# bottom right figure: k_c, k_d
ax4 = fig.add_subplot(224)
lns4 = plotfunc(['K_c', 'K_d'], colors, data, data_err, ax4)

ax4.set_ylim([0, 3e15])
ax4.set_ylabel('capital K')
ax4.set_xlabel('t')

# top left legend
lns = lns1+lns1b
labs = [l.get_label() for l in lns]
lgd1 = ax1.legend(lns, labs,
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
          ncol=3, mode="expand", borderaxespad=0.)

# top right legend
lns = lns2+lns2b
labs = [l.get_label() for l in lns]
lgd2 = ax2.legend(lns, labs,
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
          ncol=3, mode="expand", borderaxespad=0.)

# bottom left legend
lns = lns3
labs = [l.get_label() for l in lns]
lgd3 = ax3.legend(lns, labs,
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
          ncol=3, mode="expand", borderaxespad=0.)

# bottom right legend
lns = lns4
labs = [l.get_label() for l in lns]
lgd4 = ax4.legend(lns, labs,
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
          ncol=3, mode="expand", borderaxespad=0.)

axes = [ax1, ax2, ax3, ax4]

for ax, t in zip(axes, ['a)', 'b)', 'c)', 'd)']):
    ax.text(x=.04, y=.4, s=t,
            fontdict={'fontsize': 16},
            transform=ax.transAxes)


plt.tight_layout()
fig.savefig('example_trajectory_square_thesis.pdf', bbox_extra_artists=(lgd1, lgd2, lgd3, lgd4,), bbox_inches='tight')


plt.tight_layout()
fig.savefig('example_trajectory.pdf')

In [ ]:
with pd.HDFStore(path + 'X0c.h5') as store:
    trj = store.select('dat_0', where='phi=.8')
with pd.HDFStore(path + 'X0c.h5') as store:
    trj2 = store.select('dat_2', where='phi=.8')
    
trj2.index.names = trj.index.names

for trj_i in [trj, trj2]:
    trj_i.index = trj_i.index.droplevel(['eps', 'phi', 'test'])
    trj_i = trj_i.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])


# setting parameters
thresh_time = 10.
thresh_indicator = 'i_c'
thresh_value = 0.5
t_max = 40.
indicators = ['i_c', 'G']
t_scatt = [20., t_max]
cond = pd.DataFrame(index=trj.index)
cond2 = pd.DataFrame(index=trj2.index)

In [ ]:
trj2 = trj2.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])

In [ ]:
i=5
print(f'phi = 0.{i}')
with pd.HDFStore(path + 'X0c.h5') as store:
    trj = store.select('dat_0', where=f'phi=.{i}')
with pd.HDFStore(path + 'X0c.h5') as store:
    trj2 = store.select('dat_2', where=f'phi=.{i}')

trj2.index.names = trj.index.names

for trj_i in [trj, trj2]:
    trj_i.index = trj_i.index.droplevel(['eps', 'phi', 'test'])
    trj_i = trj_i.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])
trj = trj.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])
trj2 = trj2.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])
trj.head()

In [ ]:
# visualization of break even times of the clean sector.
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(4.5, 4))

for obs, color in zip(['r_c', 'r_d'], ['#179CAD', '#17AD73']):
    print(obs)
    trj_plt = trj[[obs]].unstack('sample').plot(ax=ax, color=color, legend=False, alpha=0.1)

ax.set_xlim([6, 11])
ax.set_ylim([0.071, 0.076])

ax.set_ylabel('capital return rate')
ax.set_xlabel('year')

axb = ax.twinx()
x_labels=ax.get_xticks().tolist()
ax.set_xticklabels([int(x + 2010) for x in x_labels])

tmp = trj.loc[trj['r_c']>trj['r_d']][['r_c', 'r_d', 'time']].sort_index(level='sample')
tmp.reset_index().loc[tmp.reset_index().groupby('sample')['time'].idxmin()]['time'].plot(kind='box', vert=False, 
                                                                                         ax=axb, patch_artist=True, 
                                                                                         label='', color='#C7385B')

clean_patch = mpatches.Patch(color='#179CAD', label=r'$r_c$')
dirty_patch = mpatches.Patch(color='#17AD73', label=r'$r_d$')
break_patch = mpatches.Patch(color='#C7385B', label='time of \nbreak even')
ax.legend(handles=[clean_patch, dirty_patch, break_patch])

fig.tight_layout()

fig.savefig('break_even.pdf')